<a href="https://colab.research.google.com/github/NinaNikolova/data_mining/blob/main/02_Pandas_Optimizations_Pt_2_Data_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Data Storage
#####Oптимизацията на съхранението на данни в Pandas, като разглежда различни методи за запазване на данни във файлове и анализира производителността им.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os # стандартен модул в Python, който предоставя функции за работа с файлове и директории

def print_files_and_sizes(directory):
    for root, dirs, files in os.walk(directory): #os.walk(directory) обхожда всички поддиректории и файлове в дадената директория.
        for file in files:
            file_path = os.path.join(root, file) # Получава пълния път
            file_size_bytes = os.path.getsize(file_path) #Изчислява размера в байтове
            file_size_megabytes = file_size_bytes / (1024 * 1024) # Конвертира в мегабайти
            print(f"{file_path} - {file_size_megabytes:.2f} MB") #Принтира резултата
print_files_and_sizes('data/') #  ще принтира списък с всички файлове в директорията data/ заедно с техния размер.

## Create our dataset - Създаване на Данни

In [ ]:
def get_dataset(size):
    df = pd.DataFrame() # Генерира DataFrame със случайни стойности за различни атрибути:
    df['size'] = np.random.choice(['big','medium','small'], size) # size – категория (big, medium, small).
    df['age'] = np.random.randint(1, 50, size) # age – цяло число между 1 и 50.
    df['team'] = np.random.choice(['red','blue','yellow','green'], size) # team – категория (red, blue, yellow, green).
    df['win'] = np.random.choice(['yes','no'], size) # win – дали е победа (yes, no)
    dates = pd.date_range('2020-01-01', '2022-12-31')
    df['date'] = np.random.choice(dates, size) # date – случайна дата между 2020 и 2022 г.
    df['prob'] = np.random.uniform(0, 1, size) #prob – случайна стойност между 0 и 1.
    return df

In [ ]:
def set_dtypes(df): # Преобразува типовете данни за по-добра оптимизация
    df['size'] = df['size'].astype('category') # size и team → category (за пестене на памет)
    df['team'] = df['team'].astype('category')
    df['age'] = df['age'].astype('int8') # int8 (тъй като максималната стойност е 50)
    df['prob'] = df['prob'].astype('float32') #  float32 (по-малко памет от float64).
    df['win'] = df['win'].map({'yes':True, 'no':False}) # bool (преобразува yes/no в True/False)
    return df

In [ ]:
df = get_dataset(10_000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   size    10000 non-null  object        
 1   age     10000 non-null  int32         
 2   team    10000 non-null  object        
 3   win     10000 non-null  object        
 4   date    10000 non-null  datetime64[ns]
 5   prob    10000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 429.8+ KB


In [ ]:
df = get_dataset(1_000_000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  object        
 1   age     1000000 non-null  int32         
 2   team    1000000 non-null  object        
 3   win     1000000 non-null  object        
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 42.0+ MB


In [ ]:
df = set_dtypes(df) # променя типовете на колоните на по-ефективни типове данни, за да се оптимизира използването на памет.
df.info() # Показва информация за структурата и типовете данни на df след оптимизацията.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int8          
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int8(1)
memory usage: 15.3 MB


## CSV(Comma-Separated Values) -Съхранение във файлове - Лесен за използване, но заема много памет.
Запис с и без индекс:

-Без индекс (index=False) намалява размера на файла

In [ ]:
PATH = 'data/'
PATH

'data/'

##### Запазване csv

In [ ]:
%%time
df.to_csv(PATH + 'test_csv.csv')

CPU times: total: 2.81 s
Wall time: 6.28 s


In [ ]:
print_files_and_sizes('data/') # обхожда файловете в папката data/ и отпечатва техните размери в MB.

data/test_csv.csv - 46.73 MB


In [ ]:
df = pd.read_csv(PATH + 'test_csv.csv') # Зарежда данни от CSV файл в pandas DataFrame
df.head() # извежда първите 5 реда на таблицата, за да можем бързо да видим структурата и съдържанието на данните.

,Unnamed: 0,size,age,team,win,date,prob
0,0,medium,47,yellow,False,2021-06-14,0.778592
1,1,medium,9,red,True,2021-02-11,0.201530
2,2,big,17,yellow,False,2020-12-26,0.259715
3,3,big,39,yellow,False,2020-07-19,0.753029
4,4,small,31,green,True,2022-02-23,0.029892


Запазване csv без index

In [ ]:
df = get_dataset(1_000_000) # Създава pandas DataFrame с 1 000 000 реда
df.to_csv(PATH + 'test_csv.csv', index=False) # Запазва данните в CSV файл без индекс- pandas няма да запази индекса на редовете във файла.

In [ ]:
print_files_and_sizes('data/')

data/test_csv.csv - 46.49 MB


In [ ]:
df = pd.read_csv(PATH + 'test_csv.csv')
df.head()

,size,age,team,win,date,prob
0,big,29,blue,yes,2022-03-28,0.502859
1,small,15,red,no,2022-07-24,0.290174
2,big,16,red,no,2022-01-10,0.870985
3,small,41,yellow,yes,2020-05-28,0.970990
4,small,10,green,yes,2021-10-30,0.286898


Save the csv with index and set the index column when read the file

In [ ]:
df = get_dataset(1_000_000)
df.to_csv(PATH + 'test_csv.csv', index=True) # index=True означава, че ще се запази индексът на DataFrame-а като отделна колона в CSV файла.

In [ ]:
print_files_and_sizes('data/')

data/test_csv.csv - 53.06 MB


In [ ]:
df = pd.read_csv(PATH + 'test_csv.csv', index_col=[0]) # index_col=[0] задава колоната с индекс в DataFrame. Ако CSV файлът има индексна колона като първа колона (например, 0, 1, 2, ...), тогава тази колона ще бъде автоматично разпозната като индекс на DataFrame.
df.head()

,size,age,team,win,date,prob
0,medium,22,green,yes,2021-03-23,0.638345
1,small,17,blue,no,2020-06-04,0.833034
2,medium,43,red,yes,2022-08-11,0.580966
3,medium,2,red,yes,2022-12-30,0.809185
4,big,3,green,no,2022-05-26,0.502095


### Измерване времето

In [ ]:
df = get_dataset(1_000_000)
%timeit df.to_csv(PATH + 'test_csv.csv', index=True)

6.51 s ± 52.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit df = pd.read_csv(PATH + 'test_csv.csv', index_col=[0])

1.37 s ± 55.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
df = get_dataset(1_000_000)
df = set_dtypes(df) # Извежда обобщена информация за df, включително броя на редовете, колоните, типовете на данните и използваната памет.
df.info() # Извежда информация за df, включително броя на редовете и колоните, типовете на данните и използваната памет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int8          
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int8(1)
memory usage: 15.3 MB


In [ ]:
df.to_csv(PATH + 'test_csv.csv') # Записва df в CSV файл с име test_csv.csv в директорията, посочена в PATH.  CSV (Comma-Separated Values) е текстов формат за съхранение на таблици с данни.
df = pd.read_csv(PATH + 'test_csv.csv', index_col=[0]) # Зарежда обратно данните от файла test_csv.csv в DataFrame df. index_col=[0] означава, че първата колона ще бъде използвана като индекс на DataFrame-
df.info() # Извежда информация за df, включително броя на редовете и колоните, типовете на данните и използваната памет.

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   size    1000000 non-null  object 
 1   age     1000000 non-null  int64  
 2   team    1000000 non-null  object 
 3   win     1000000 non-null  bool   
 4   date    1000000 non-null  object 
 5   prob    1000000 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 46.7+ MB


In [ ]:
df = pd.read_csv(PATH + 'test_csv.csv', index_col=[0],
                 dtype={'size':'category',
                        'age':'int8',
                        'team':'category'

                 }) # dtype={...} определя типовете на данните за конкретни колони, за да подобри ефективността.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   size    1000000 non-null  category
 1   age     1000000 non-null  int8    
 2   team    1000000 non-null  category
 3   win     1000000 non-null  bool    
 4   date    1000000 non-null  object  
 5   prob    1000000 non-null  float64 
dtypes: bool(1), category(2), float64(1), int8(1), object(1)
memory usage: 26.7+ MB


## Pickle - Двоичен формат за бързо зареждане и запис. Запазва типовете данни, но е специфичен за Python.
- 38.79 MB (15.3 MB)
- 1.77s to write
- 722ms to read

In [ ]:
df = get_dataset(1_000_000)
%timeit df.to_pickle(PATH + 'test.pickle') #  Записва df във файл test.pickle с помощта на pickle (бинарен формат за сериализация в Python)
%timeit df_pickle = pd.read_pickle(PATH + 'test.pickle') # %timeit измерва колко време отнема тази операция при многократно изпълнение и връща средното време.

1.77 s ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
722 ms ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print_files_and_sizes('data/')

data/test.pickle - 38.79 MB
data/test_csv.csv - 46.73 MB


In [ ]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
df.to_pickle(PATH + 'test.pickle')
df_pickle = pd.read_pickle(PATH + 'test.pickle')
df_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int8          
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int8(1)
memory usage: 15.3 MB


## Parquet - Колонно-базиран формат, идеален за големи обеми данни.По-малък размер. Поддържа се от различни среди, включително Apache Spark.
```
!pip install pyarrow
!pip install fastparquet

In [ ]:
df = get_dataset(1_000_000)
%timeit df.to_parquet(PATH + 'test.parquet')
%timeit df_parquet = pd.read_parquet(PATH + 'test.parquet')

962 ms ± 44.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
386 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print_files_and_sizes('data/')

data/test.parquet - 10.54 MB
data/test.pickle - 15.26 MB
data/test_csv.csv - 46.73 MB


In [ ]:
pd.read_parquet(PATH + 'test.parquet', columns=['date', 'win'])

,date,win
0,2020-03-11,no
1,2020-01-03,no
2,2022-01-23,yes
3,2020-05-23,no
4,2022-11-26,no
...,...,...
999995,2021-01-02,yes
999996,2021-07-10,no
999997,2022-02-12,no
999998,2021-05-11,no


## Feather-Подходящ за краткосрочно съхранение и бърза обработка. Не поддържа компресия.

In [ ]:
df = get_dataset(1_000_000)
%timeit df.to_feather(PATH + 'test.feather')
%timeit df_feather = pd.read_feather(PATH + 'test.feather')

540 ms ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
321 ms ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print_files_and_sizes('data/')

data/test.feather - 27.72 MB
data/test.parquet - 10.54 MB
data/test.pickle - 15.26 MB
data/test_csv.csv - 46.73 MB


##### Parquet е най-добрият вариант за дългосрочно съхранение, защото е компактен и бърз.

##### Feather е подходящ за бързи операции, но не е най-оптималният за дългосрочно съхранение.

##### Pickle е удобен за Python, но не се поддържа извън екосистемата на Python.

##### CSV е лесен за използване, но заема много памет и е най-бавен.

## Other

In [ ]:
df = get_dataset(1_000_000) # създава DataFrame с 1 000 000 реда.
df.to_ # Започва използване на метод за записване на DataFrame в файл, но не е завършено.

df.to_csv('filename.csv') → Записва като CSV файл

df.to_excel('filename.xlsx') → Записва като Excel файл

df.to_pickle('filename.pkl') → Записва като Pickle файл (по-бърз за четене/писане)

df.to_parquet('filename.parquet') → Записва като Parquet (ефективен за големи данни)

df.to_feather('filename.feather') → Записва като Feather (по-бърз за четене)